In [1]:
import sys
sys.path.append('../')

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

from utils.utils import Utils
from utils.preprocess import Preprocess

from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Reshape, Concatenate
from tensorflow.keras import optimizers
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [4]:
utils = Utils()
pre = Preprocess('clean')

In [5]:
train_data = utils.read_data("../dataset/train/train-final.csv")
print(train_data.shape)
dev_data = utils.read_data("../dataset/dev/dev-final.csv")
print(dev_data.shape)

(1860, 317)
(270, 304)


In [6]:
train_data['age_bin'] = train_data.age.apply(lambda x: utils.categorize_age(x))
train_data[['age', 'age_bin']]

,age,age_bin
0,33,1
1,33,1
2,33,1
3,33,1
4,33,1
...,...,...
1855,34,1
1856,34,1
1857,34,1
1858,34,1


In [7]:
dev_data['age_bin'] = dev_data.age.apply(lambda x: utils.categorize_age(x))
dev_data[['age', 'age_bin']]

,age,age_bin
0,20,0
1,20,0
2,20,0
3,20,0
4,20,0
...,...,...
265,37,1
266,37,1
267,37,1
268,37,1


In [8]:
def prepare_data(train_df, dev_df, features=['gender', 'education', 'race', 'age'], target='gold_empathy'):
    x_train = []
    x_dev = []
    for f in features:
        label_encoder = LabelEncoder()
        one_hot_encoder = OneHotEncoder(sparse=False)
        
        train_feature_data = 0
        dev_feature_data = 0
        
        train_feature_data = label_encoder.fit_transform(train_df[f].values) 
        train_feature_data = np.reshape(train_feature_data, (len(train_feature_data), 1))
        train_feature_data = one_hot_encoder.fit_transform(train_feature_data)
        x_train.append(train_feature_data)   
        
        dev_feature_data = label_encoder.transform(dev_df[f].values) 
        dev_feature_data = np.reshape(dev_feature_data, (len(dev_feature_data), 1))
        dev_feature_data = one_hot_encoder.transform(dev_feature_data)
        x_dev.append(dev_feature_data)
    
    y_train = np.reshape(train_df[target].values.tolist(), (len(train_df[target].values.tolist()), 1))
    y_dev = np.reshape(dev_df[target].values.tolist(), (len(dev_df[target].values.tolist()), 1))
    return x_train, y_train, x_dev, y_dev

In [13]:
features=['gender', 'education', 'race', 'age_bin', 'gold_emotion']

In [14]:
x_train, y_train, x_dev, y_dev = prepare_data(train_data, 
                                              dev_data,
                                              features=features,
                                              target='gold_empathy')
print(y_train.shape)
print(len(x_train))
print(x_train[0].shape)
print(x_train[1].shape)
print(x_train[2].shape)
print(x_train[3].shape)
print(x_train[4].shape)

(1860, 1)
5
(1860, 3)
(1860, 6)
(1860, 6)
(1860, 4)
(1860, 7)


In [20]:
def build_model(input_feature):
    inputs = Input(shape=(input_feature.shape[1]))
    embedding = Embedding(input_dim=input_feature.shape[1], output_dim=3, trainable=True)(inputs)
    x = Flatten()(embedding)
    out = Dense(1)(x)

    model = Model(inputs=inputs, outputs=out)
    model.compile(optimizer=optimizers.Adam(lr=0.001), loss='mse')
    model.summary()
    return model

In [61]:
i = 0
myModel = build_model(x_train[i])

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 3, 3)              9         
_________________________________________________________________
flatten_7 (Flatten)          (None, 9)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 10        
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________


In [62]:
checkpoint_filepath = "../entity-embeddings/models/" + features[i] + "-epoch-{epoch:02d}-val-loss-{val_loss:02f}.h5"
print(checkpoint_filepath)
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                            save_weights_only=True,
                                            monitor='val_loss',
                                            mode='min',
                                            save_freq = 'epoch',
                                            save_best_only=True)

../entity-embeddings/models/gender-epoch-{epoch:02d}-val-loss-{val_loss:02f}.h5


In [63]:
myModel.fit(x_train[i],
            y_train,
            epochs=200, 
            batch_size=32, 
            verbose=1,
            shuffle=True,
            validation_data=(x_dev[i], y_dev),
           callbacks=[model_checkpoint_callback])

Epoch 1/200
59/59 [==============================] - 0s 2ms/step - loss: 16.5983 - val_loss: 15.2471
Epoch 2/200
59/59 [==============================] - 0s 1ms/step - loss: 15.4790 - val_loss: 13.8615
Epoch 3/200
59/59 [==============================] - 0s 2ms/step - loss: 13.6128 - val_loss: 11.6645
Epoch 4/200
59/59 [==============================] - 0s 2ms/step - loss: 11.0470 - val_loss: 9.0667
Epoch 5/200
59/59 [==============================] - 0s 1ms/step - loss: 8.3299 - val_loss: 6.6766
Epoch 6/200
59/59 [==============================] - 0s 2ms/step - loss: 6.0749 - val_loss: 4.9303
Epoch 7/200
59/59 [==============================] - 0s 1ms/step - loss: 4.5681 - val_loss: 3.9708
Epoch 8/200
59/59 [==============================] - 0s 2ms/step - loss: 3.7699 - val_loss: 3.5721
Epoch 9/200
59/59 [==============================] - 0s 1ms/step - loss: 3.4532 - val_loss: 3.4706
Epoch 10/200
59/59 [==============================] - 0s 2ms/step - loss: 3.3610 - val_loss: 3.4627
Ep

59/59 [==============================] - 0s 889us/step - loss: 3.3330 - val_loss: 3.4796
Epoch 83/200
59/59 [==============================] - 0s 893us/step - loss: 3.3330 - val_loss: 3.4685
Epoch 84/200
59/59 [==============================] - 0s 816us/step - loss: 3.3327 - val_loss: 3.4799
Epoch 85/200
59/59 [==============================] - 0s 994us/step - loss: 3.3338 - val_loss: 3.4808
Epoch 86/200
59/59 [==============================] - 0s 904us/step - loss: 3.3333 - val_loss: 3.4761
Epoch 87/200
59/59 [==============================] - 0s 1ms/step - loss: 3.3351 - val_loss: 3.4608
Epoch 88/200
59/59 [==============================] - 0s 1ms/step - loss: 3.3335 - val_loss: 3.4703
Epoch 89/200
59/59 [==============================] - 0s 1ms/step - loss: 3.3337 - val_loss: 3.4744
Epoch 90/200
59/59 [==============================] - 0s 1ms/step - loss: 3.3342 - val_loss: 3.4711
Epoch 91/200
59/59 [==============================] - 0s 1ms/step - loss: 3.3332 - val_loss: 3.4784
Epo

59/59 [==============================] - 0s 837us/step - loss: 3.3317 - val_loss: 3.4680
Epoch 163/200
59/59 [==============================] - 0s 860us/step - loss: 3.3334 - val_loss: 3.4764
Epoch 164/200
59/59 [==============================] - 0s 1ms/step - loss: 3.3318 - val_loss: 3.4715
Epoch 165/200
59/59 [==============================] - 0s 882us/step - loss: 3.3323 - val_loss: 3.4728
Epoch 166/200
59/59 [==============================] - 0s 983us/step - loss: 3.3347 - val_loss: 3.4719
Epoch 167/200
59/59 [==============================] - 0s 820us/step - loss: 3.3336 - val_loss: 3.4661
Epoch 168/200
59/59 [==============================] - 0s 827us/step - loss: 3.3327 - val_loss: 3.4630
Epoch 169/200
59/59 [==============================] - 0s 832us/step - loss: 3.3324 - val_loss: 3.4797
Epoch 170/200
59/59 [==============================] - 0s 866us/step - loss: 3.3382 - val_loss: 3.4777
Epoch 171/200
59/59 [==============================] - 0s 1ms/step - loss: 3.3337 - val_l

In [58]:
myModel.load_weights("../entity-embeddings/models/race-epoch-133-val-loss-3.388175.h5")
embs = myModel.layers[1].get_weights()
embs

[array([[-0.41480175,  0.3263364 ,  0.24606583],
        [-0.08021391,  0.33736658,  0.5623005 ],
        [-0.02836956, -0.02215928,  0.02009453],
        [ 0.04501821, -0.00250716, -0.02140896],
        [ 0.03879731, -0.03588603,  0.04247056],
        [-0.0134331 ,  0.04927707,  0.04220125]], dtype=float32)]

In [59]:
embs[0].shape

(6, 3)

In [60]:
np.savetxt('../entity-embeddings/gender-embeddings.txt', embs[0])

In [67]:
data = np.loadtxt('../entity-embeddings/gender-embeddings.txt')
data.shape

(3, 3)